In [4]:


import pandas as pd


def deserialize(pz):
    if os.path.isfile(pz):
        compressed = open(pz, 'rb').read()
        pkl = zlib.decompress(compressed)
        tpl = pickle.loads(pkl)
        return tpl
    else:
        return []


lubm_classes_file = "../data/lubm1_intact/encoding/lubm_classes.pz"
lubm_properties_file = "../data/lubm1_intact/encoding/lubm_properties.pz"
lubm_properties_groups_file = "../data/lubm1_intact/encoding/lubm_properties_groups.pz"
tbox_file = "../data/lubm1_intact/encoding/tbox.nt"

lubm_classes = deserialize(lubm_classes_file)
lubm_properties = deserialize(lubm_properties_file)
lubm_properties_groups = deserialize(lubm_properties_groups_file)

data = []

for key, value in lubm_classes.items():
    data.append({"type": "class", "uri": key, "index": value})

for key, value in lubm_properties.items():
    data.append({"type": "property", "uri": key, "index": value})

for key, value in lubm_properties_groups.items():
    data.append({"type": "property_group", "uri": key, "index": value})

df = pd.DataFrame(data)
print(df[df.type == "class"].size,
      df[df.type == "property"].size,
      df[df.type == "property_group"].size)

#lubm_classes
df.shape

NameError: name 'os' is not defined

# Creating a DF with clases, properties, property_groups
Even though relative - there is no relation between index

In [ ]:
result_df = df.pivot_table(index='index', columns='type', values='uri', aggfunc='first')

In [ ]:
result_df

In [ ]:
df_class = df[df.type == "class"]
df_property = df[df.type == "property"]
df_property_group = df[df.type == "property_group"]

In [ ]:
col_names = ["subject", "predicate", "object"]
#df2 = pd.read_csv(tbox_file, delimiter=' ', names=col_names)
df2 = pd.read_csv(tbox_file, delimiter=' ')
#df2['object'] = df2['object'].apply(lambda x: x.rstrip('.'))
df2 = df2.drop(df2.columns[3], axis=1)
df2.columns = col_names
df2.shape

# I want to see what "property types" are in the #type in df2 for "df1" values marked as "property"

In [ ]:
df_class

In [ ]:
df_property_group

# I will seperate only the subject #end part and compare

In [ ]:
df_property_copy = df_property.copy()

In [ ]:
df_property_copy.uri = df_property_copy.uri.str.extract(r'#(.+)')

In [ ]:
df_property_copy

In [ ]:
df1_property_list = df_property_copy.uri.to_list()

# DF 2 CLASSES

In [ ]:
df2

In [ ]:
df2_types = df2[df2.predicate.str.contains('#type')]

In [ ]:
df2_classes = df2[df2.object.str.contains('#Class')]

In [ ]:
df2_classes

# DF 2 PROPERTIES

In [ ]:
df2_types

# Compare property list from df1 to df2 subject, and get object "type"

In [ ]:
df1_property_list

In [ ]:
types = []
for x in range(len(df1_property_list)):
    types.append(df2_types[df2_types.subject.str.contains(df1_property_list[x])].object.str.extract(r'#(.+)'))

In [ ]:
types_2 = []
for type in types:
    types_2.append(type.to_string().strip())

In [ ]:
types_2

In [ ]:
df2_properties = df2_types.copy()

In [ ]:
df2_properties = df2_properties[df2_properties.object.str.contains("Property")].subject.unique()

In [ ]:
df2_properties

# DF2 SUB PROPERTIES

In [ ]:
df2_sub_properties = df2.copy()

In [ ]:
df2_sub_properties = df2_sub_properties[df2_sub_properties.predicate.str.contains("subPropertyOf")]

In [ ]:
df2_sub_properties

In [ ]:



def get_lubm_graph_type(s):
    s = s.split('/')[-1]
    print(s)
    s = s.split('_')[-1]
    print(s)
    m = re.search("\d", s)
    print(m)
    return s[:m.start()]

In [ ]:
get_lubm_graph_type("HTTP_www.Department0.University0.edu_AssistantProfessor8_Publication1.nt")


In [ ]:
get_lubm_graph_type(
    "C:/Users/martt/PycharmProjects/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")

In [36]:
from rdflib import Graph
import codecs
import os

In [37]:
input_graph = Graph()
input_graph2 = Graph()
input_graph3 = Graph()
inference_graph = Graph()

input_file = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu.nt")
input_file2 = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")

input_file3 = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0_Publication0.nt")

inference_file = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/jena_inference_with_descriptions/HTTP_www.Department0.University0.edu.nt")

input_graph.parse(data=codecs.open(input_file, encoding="UTF-8").read(), format="nt")
inference_graph.parse(data=codecs.open(inference_file, encoding="UTF-8").read(), format="nt")
input_graph2.parse(data=codecs.open(input_file2, encoding="UTF-8").read(), format="nt")
input_graph3.parse(data=codecs.open(input_file3, encoding="UTF-8").read(), format="nt")
graph = input_graph

In [38]:
triples_list = sorted(list(input_graph))[0]

In [39]:
from graph_words_encoder import GraphWordsEncoder
from prepare_data_utils import get_properties_dict, get_classes_dict, get_properties_groups

In [40]:
import configparser
import json
import logging

In [41]:
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

config = configparser.ConfigParser()
with open('../config.json', 'r') as f:
    config = json.load(f)

In [42]:
LUBM_ONTOLOGY_ENDPOINT = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT']
LUBM_ONTOLOGY_REPOSITORY = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_REPOSITORY']
LUBM_ONTOLOGY_ENDPOINT_TYPE = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT_TYPE']
PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['PROPERTIES_QUERY']
QUERY_LIMIT = config['LUBM']['SPARQL_PARAMS']['QUERY_LIMIT']
PROPERTIES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_DICT_FILE']
CLASSES_QUERY = config['LUBM']['SPARQL_PARAMS']['CLASSES_QUERY']
LUBM_ENCODING_DIRECTORY = config['LUBM']['ENCODING_PARAMS']['ENCODING_DIRECTORY']
CLASSES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['CLASSES_DICT_FILE']
PROPERTIES_GROUPS_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_GROUPS_FILE']
GRAPH_WORDS_ENCODER = config['LUBM']['ENCODING_PARAMS']['GRAPH_WORDS_ENCODER']
SUB_PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['SUB_PROPERTIES_QUERY']
INPUT_GRAPHS_FOLDER = config['LUBM']['DATA']['INPUT_GRAPHS_FOLDER']
INFERENCE_GRAPHS_FOLDER = config['LUBM']['DATA']['INFERENCE_GRAPHS_FOLDER']
DATASET_ENCODING_FILE = config['LUBM']['ENCODING_PARAMS']['DATASET_ENCODING_FILE']
INPUT_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INPUT_GRAPH_WORDS_CATALOGUE_FILE']
INFERENCE_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INFERENCE_GRAPH_WORDS_CATALOGUE_FILE']
EMBEDDING_MATRIX_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['EMBEDDING_MATRIX_CATALOGUE_FILE']
MAXIMUM_MATRIX_SIZE = config['LUBM']['ENCODING_PARAMS']['MAXIMUM_MATRIX_SIZE']
HOPE_EMBEDDING_SIZE = config['LUBM']['ENCODING_PARAMS']['HOPE_EMBEDDING_SIZE']

In [43]:
properties_dict = get_properties_dict(LUBM_ENCODING_DIRECTORY + PROPERTIES_DICT_FILE, PROPERTIES_QUERY,
                                      LUBM_ONTOLOGY_REPOSITORY, LUBM_ONTOLOGY_ENDPOINT,
                                      LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [44]:
classes_dict = get_classes_dict(LUBM_ENCODING_DIRECTORY + CLASSES_DICT_FILE, CLASSES_QUERY, LUBM_ONTOLOGY_REPOSITORY,
                                LUBM_ONTOLOGY_ENDPOINT,
                                LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [45]:
properties_group_dict = get_properties_groups(LUBM_ENCODING_DIRECTORY + PROPERTIES_GROUPS_FILE, SUB_PROPERTIES_QUERY,
                                              LUBM_ONTOLOGY_REPOSITORY,
                                              LUBM_ONTOLOGY_ENDPOINT,
                                              LUBM_ONTOLOGY_ENDPOINT_TYPE, properties_dict, QUERY_LIMIT)

In [48]:
for key in properties_dict.getKeys():
    print(key)

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://swat.cse.lehigh.edu/onto/univ-bench.owl#advisor
http://swat.cse.lehigh.edu/onto/univ-bench.owl#affiliateOf
http://swat.cse.lehigh.edu/onto/univ-bench.owl#affiliatedOrganizationOf
http://swat.cse.lehigh.edu/onto/univ-bench.owl#age
http://swat.cse.lehigh.edu/onto/univ-bench.owl#degreeFrom
http://swat.cse.lehigh.edu/onto/univ-bench.owl#doctoralDegreeFrom
http://swat.cse.lehigh.edu/onto/univ-bench.owl#emailAddress
http://swat.cse.lehigh.edu/onto/univ-bench.owl#hasAlumnus
http://swat.cse.lehigh.edu/onto/univ-bench.owl#headOf
http://swat.cse.lehigh.edu/onto/univ-bench.owl#listedCourse
http://swat.cse.lehigh.edu/onto/univ-bench.owl#mastersDegreeFrom
http://swat.cse.lehigh.edu/onto/univ-bench.owl#member
http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf
http://swat.cse.lehigh.edu/onto/univ-bench.owl#name
http://swat.cse.lehigh.edu/onto/univ-bench.owl#orgPublication
http://swat.cse.lehigh.edu/onto/univ-bench.owl#publicationAuthor
http:

In [27]:
lubm_graph_words_encoder = GraphWordsEncoder(properties_dict, properties_group_dict, classes_dict)

In [28]:
input_graph_encoding, resources_dictionary = lubm_graph_words_encoder.encode_graph(input_graph)

http://www.Department0.University0.edu http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://swat.cse.lehigh.edu/onto/univ-bench.owl#Department
-1 1 1

http://www.Department0.University0.edu/FullProfessor7 http://swat.cse.lehigh.edu/onto/univ-bench.owl#headOf http://www.Department0.University0.edu
-1 2 -2

http://www.Department0.University0.edu/GraduateStudent0 http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf http://www.Department0.University0.edu
-3 3 -2

http://www.Department0.University0.edu/GraduateStudent1 http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf http://www.Department0.University0.edu
-4 3 -2

http://www.Department0.University0.edu/GraduateStudent10 http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf http://www.Department0.University0.edu
-5 3 -2

http://www.Department0.University0.edu/GraduateStudent100 http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf http://www.Department0.University0.edu
-6 3 -2

http://www.Department0.University0.edu/GraduateS

In [29]:
resources_dictionary.keys()

dict_keys([4, 2, 13, 22])

In [30]:
inference_graph_encoding, resources_dictionary = lubm_graph_words_encoder.encode_graph(inference_graph, inference=True)

KeyError: 4

In [31]:
resources_dictionary.keys()

dict_keys([4, 2, 13, 22])

In [35]:
for key in properties_dict.getKeys():
    print(key)

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://swat.cse.lehigh.edu/onto/univ-bench.owl#advisor
http://swat.cse.lehigh.edu/onto/univ-bench.owl#affiliateOf
http://swat.cse.lehigh.edu/onto/univ-bench.owl#affiliatedOrganizationOf
http://swat.cse.lehigh.edu/onto/univ-bench.owl#age
http://swat.cse.lehigh.edu/onto/univ-bench.owl#degreeFrom
http://swat.cse.lehigh.edu/onto/univ-bench.owl#doctoralDegreeFrom
http://swat.cse.lehigh.edu/onto/univ-bench.owl#emailAddress
http://swat.cse.lehigh.edu/onto/univ-bench.owl#hasAlumnus
http://swat.cse.lehigh.edu/onto/univ-bench.owl#headOf
http://swat.cse.lehigh.edu/onto/univ-bench.owl#listedCourse
http://swat.cse.lehigh.edu/onto/univ-bench.owl#mastersDegreeFrom
http://swat.cse.lehigh.edu/onto/univ-bench.owl#member
http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf
http://swat.cse.lehigh.edu/onto/univ-bench.owl#name
http://swat.cse.lehigh.edu/onto/univ-bench.owl#orgPublication
http://swat.cse.lehigh.edu/onto/univ-bench.owl#publicationAuthor
http:

In [ ]:
for key in properties_group_dict.keys():  #%%
    pass

import pandas as pd


def deserialize(pz):
    if os.path.isfile(pz):
        compressed = open(pz, 'rb').read()
        pkl = zlib.decompress(compressed)
        tpl = pickle.loads(pkl)
        return tpl
    else:
        return []


lubm_classes_file = "../data/lubm1_intact/encoding/lubm_classes.pz"
lubm_properties_file = "../data/lubm1_intact/encoding/lubm_properties.pz"
lubm_properties_groups_file = "../data/lubm1_intact/encoding/lubm_properties_groups.pz"
tbox_file = "../data/lubm1_intact/encoding/tbox.nt"

lubm_classes = deserialize(lubm_classes_file)
lubm_properties = deserialize(lubm_properties_file)
lubm_properties_groups = deserialize(lubm_properties_groups_file)

data = []

for key, value in lubm_classes.items():
    data.append({"type": "class", "uri": key, "index": value})

for key, value in lubm_properties.items():
    data.append({"type": "property", "uri": key, "index": value})

for key, value in lubm_properties_groups.items():
    data.append({"type": "property_group", "uri": key, "index": value})

df = pd.DataFrame(data)
print(df[df.type == "class"].size,
      df[df.type == "property"].size,
      df[df.type == "property_group"].size)

#lubm_classes
df.shape

# Creating a DF with clases, properties, property_groups
Even though relative - there is no relation between index

In [ ]:
result_df = df.pivot_table(index='index', columns='type', values='uri', aggfunc='first')

In [ ]:
result_df

In [ ]:
df_class = df[df.type == "class"]
df_property = df[df.type == "property"]
df_property_group = df[df.type == "property_group"]

In [ ]:
col_names = ["subject", "predicate", "object"]
#df2 = pd.read_csv(tbox_file, delimiter=' ', names=col_names)
df2 = pd.read_csv(tbox_file, delimiter=' ')
#df2['object'] = df2['object'].apply(lambda x: x.rstrip('.'))
df2 = df2.drop(df2.columns[3], axis=1)
df2.columns = col_names
df2.shape

# I want to see what "property types" are in the #type in df2 for "df1" values marked as "property"

In [ ]:
df_class

In [ ]:
df_property_group

# I will seperate only the subject #end part and compare

In [ ]:
df_property_copy = df_property.copy()

In [ ]:
df_property_copy.uri = df_property_copy.uri.str.extract(r'#(.+)')

In [ ]:
df_property_copy

In [ ]:
df1_property_list = df_property_copy.uri.to_list()

# DF 2 CLASSES

In [ ]:
df2

In [ ]:
df2_types = df2[df2.predicate.str.contains('#type')]

In [ ]:
df2_classes = df2[df2.object.str.contains('#Class')]

In [ ]:
df2_classes

# DF 2 PROPERTIES

In [ ]:
df2_types

# Compare property list from df1 to df2 subject, and get object "type"

In [ ]:
df1_property_list

In [ ]:
types = []
for x in range(len(df1_property_list)):
    types.append(df2_types[df2_types.subject.str.contains(df1_property_list[x])].object.str.extract(r'#(.+)'))

In [ ]:
types_2 = []
for type in types:
    types_2.append(type.to_string().strip())

In [ ]:
types_2

In [ ]:
df2_properties = df2_types.copy()

In [ ]:
df2_properties = df2_properties[df2_properties.object.str.contains("Property")].subject.unique()

In [ ]:
df2_properties

# DF2 SUB PROPERTIES

In [ ]:
df2_sub_properties = df2.copy()

In [ ]:
df2_sub_properties = df2_sub_properties[df2_sub_properties.predicate.str.contains("subPropertyOf")]

In [ ]:
df2_sub_properties

In [ ]:



def get_lubm_graph_type(s):
    s = s.split('/')[-1]
    print(s)
    s = s.split('_')[-1]
    print(s)
    m = re.search("\d", s)
    print(m)
    return s[:m.start()]

In [ ]:
get_lubm_graph_type("HTTP_www.Department0.University0.edu_AssistantProfessor8_Publication1.nt")


In [ ]:
get_lubm_graph_type(
    "C:/Users/martt/PycharmProjects/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")

In [ ]:
from rdflib import Graph
import codecs
import os

In [ ]:
input_graph = Graph()
input_graph2 = Graph()
input_graph3 = Graph()
inference_graph = Graph()

input_file = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu.nt")
input_file2 = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")

input_file3 = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0_Publication0.nt")

inference_file = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/jena_inference_with_descriptions/HTTP_www.Department0.University0.edu.nt")

input_graph.parse(data=codecs.open(input_file, encoding="UTF-8").read(), format="nt")
inference_graph.parse(data=codecs.open(inference_file, encoding="UTF-8").read(), format="nt")
input_graph2.parse(data=codecs.open(input_file2, encoding="UTF-8").read(), format="nt")
input_graph3.parse(data=codecs.open(input_file3, encoding="UTF-8").read(), format="nt")
graph = input_graph

In [ ]:
triples_list = sorted(list(input_graph))[0]

In [ ]:
from graph_words_encoder import GraphWordsEncoder
from prepare_data_utils import get_properties_dict, get_classes_dict, get_properties_groups

In [ ]:
import configparser
import json
import logging

In [ ]:
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

config = configparser.ConfigParser()
with open('../config.json', 'r') as f:
    config = json.load(f)

In [ ]:
LUBM_ONTOLOGY_ENDPOINT = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT']
LUBM_ONTOLOGY_REPOSITORY = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_REPOSITORY']
LUBM_ONTOLOGY_ENDPOINT_TYPE = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT_TYPE']
PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['PROPERTIES_QUERY']
QUERY_LIMIT = config['LUBM']['SPARQL_PARAMS']['QUERY_LIMIT']
PROPERTIES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_DICT_FILE']
CLASSES_QUERY = config['LUBM']['SPARQL_PARAMS']['CLASSES_QUERY']
LUBM_ENCODING_DIRECTORY = config['LUBM']['ENCODING_PARAMS']['ENCODING_DIRECTORY']
CLASSES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['CLASSES_DICT_FILE']
PROPERTIES_GROUPS_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_GROUPS_FILE']
GRAPH_WORDS_ENCODER = config['LUBM']['ENCODING_PARAMS']['GRAPH_WORDS_ENCODER']
SUB_PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['SUB_PROPERTIES_QUERY']
INPUT_GRAPHS_FOLDER = config['LUBM']['DATA']['INPUT_GRAPHS_FOLDER']
INFERENCE_GRAPHS_FOLDER = config['LUBM']['DATA']['INFERENCE_GRAPHS_FOLDER']
DATASET_ENCODING_FILE = config['LUBM']['ENCODING_PARAMS']['DATASET_ENCODING_FILE']
INPUT_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INPUT_GRAPH_WORDS_CATALOGUE_FILE']
INFERENCE_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INFERENCE_GRAPH_WORDS_CATALOGUE_FILE']
EMBEDDING_MATRIX_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['EMBEDDING_MATRIX_CATALOGUE_FILE']
MAXIMUM_MATRIX_SIZE = config['LUBM']['ENCODING_PARAMS']['MAXIMUM_MATRIX_SIZE']
HOPE_EMBEDDING_SIZE = config['LUBM']['ENCODING_PARAMS']['HOPE_EMBEDDING_SIZE']

In [ ]:
properties_dict = get_properties_dict(LUBM_ENCODING_DIRECTORY + PROPERTIES_DICT_FILE, PROPERTIES_QUERY,
                                      LUBM_ONTOLOGY_REPOSITORY, LUBM_ONTOLOGY_ENDPOINT,
                                      LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [ ]:
classes_dict = get_classes_dict(LUBM_ENCODING_DIRECTORY + CLASSES_DICT_FILE, CLASSES_QUERY, LUBM_ONTOLOGY_REPOSITORY,
                                LUBM_ONTOLOGY_ENDPOINT,
                                LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [ ]:
properties_group_dict = get_properties_groups(LUBM_ENCODING_DIRECTORY + PROPERTIES_GROUPS_FILE, SUB_PROPERTIES_QUERY,
                                              LUBM_ONTOLOGY_REPOSITORY,
                                              LUBM_ONTOLOGY_ENDPOINT,
                                              LUBM_ONTOLOGY_ENDPOINT_TYPE, properties_dict, QUERY_LIMIT)

In [ ]:
for key in properties_dict.getKeys():
    print(key)

In [ ]:
lubm_graph_words_encoder = GraphWordsEncoder(properties_dict, properties_group_dict, classes_dict)

In [ ]:
input_graph_encoding, resources_dictionary = lubm_graph_words_encoder.encode_graph(input_graph)

In [ ]:
resources_dictionary.keys()

In [ ]:
inference_graph_encoding, resources_dictionary = lubm_graph_words_encoder.encode_graph(inference_graph, inference=True)

In [ ]:
resources_dictionary.keys()

In [ ]:
for key in properties_dict.getKeys():
    print(key)

In [ ]:
for key in properties_group_dict.keys():  #%%

import pandas as pd


def deserialize(pz):
    if os.path.isfile(pz):
        compressed = open(pz, 'rb').read()
        pkl = zlib.decompress(compressed)
        tpl = pickle.loads(pkl)
        return tpl
    else:
        return []


lubm_classes_file = "../data/lubm1_intact/encoding/lubm_classes.pz"
lubm_properties_file = "../data/lubm1_intact/encoding/lubm_properties.pz"
lubm_properties_groups_file = "../data/lubm1_intact/encoding/lubm_properties_groups.pz"
tbox_file = "../data/lubm1_intact/encoding/tbox.nt"

lubm_classes = deserialize(lubm_classes_file)
lubm_properties = deserialize(lubm_properties_file)
lubm_properties_groups = deserialize(lubm_properties_groups_file)

data = []

for key, value in lubm_classes.items():
    data.append({"type": "class", "uri": key, "index": value})

for key, value in lubm_properties.items():
    data.append({"type": "property", "uri": key, "index": value})

for key, value in lubm_properties_groups.items():
    data.append({"type": "property_group", "uri": key, "index": value})

df = pd.DataFrame(data)
print(df[df.type == "class"].size,
      df[df.type == "property"].size,
      df[df.type == "property_group"].size)

#lubm_classes
df.shape

# Creating a DF with clases, properties, property_groups
Even though relative - there is no relation between index

In [ ]:
result_df = df.pivot_table(index='index', columns='type', values='uri', aggfunc='first')

In [ ]:
result_df

In [ ]:
df_class = df[df.type == "class"]
df_property = df[df.type == "property"]
df_property_group = df[df.type == "property_group"]

In [ ]:
col_names = ["subject", "predicate", "object"]
#df2 = pd.read_csv(tbox_file, delimiter=' ', names=col_names)
df2 = pd.read_csv(tbox_file, delimiter=' ')
#df2['object'] = df2['object'].apply(lambda x: x.rstrip('.'))
df2 = df2.drop(df2.columns[3], axis=1)
df2.columns = col_names
df2.shape

# I want to see what "property types" are in the #type in df2 for "df1" values marked as "property"

In [ ]:
df_class

In [ ]:
df_property_group

# I will seperate only the subject #end part and compare

In [ ]:
df_property_copy = df_property.copy()

In [ ]:
df_property_copy.uri = df_property_copy.uri.str.extract(r'#(.+)')

In [ ]:
df_property_copy

In [ ]:
df1_property_list = df_property_copy.uri.to_list()

# DF 2 CLASSES

In [ ]:
df2

In [ ]:
df2_types = df2[df2.predicate.str.contains('#type')]

In [ ]:
df2_classes = df2[df2.object.str.contains('#Class')]

In [ ]:
df2_classes

# DF 2 PROPERTIES

In [ ]:
df2_types

# Compare property list from df1 to df2 subject, and get object "type"

In [ ]:
df1_property_list

In [ ]:
types = []
for x in range(len(df1_property_list)):
    types.append(df2_types[df2_types.subject.str.contains(df1_property_list[x])].object.str.extract(r'#(.+)'))

In [ ]:
types_2 = []
for type in types:
    types_2.append(type.to_string().strip())

In [ ]:
types_2

In [ ]:
df2_properties = df2_types.copy()

In [ ]:
df2_properties = df2_properties[df2_properties.object.str.contains("Property")].subject.unique()

In [ ]:
df2_properties

# DF2 SUB PROPERTIES

In [ ]:
df2_sub_properties = df2.copy()

In [ ]:
df2_sub_properties = df2_sub_properties[df2_sub_properties.predicate.str.contains("subPropertyOf")]

In [ ]:
df2_sub_properties

In [ ]:



def get_lubm_graph_type(s):
    s = s.split('/')[-1]
    print(s)
    s = s.split('_')[-1]
    print(s)
    m = re.search("\d", s)
    print(m)
    return s[:m.start()]

In [ ]:
get_lubm_graph_type("HTTP_www.Department0.University0.edu_AssistantProfessor8_Publication1.nt")


In [ ]:
get_lubm_graph_type(
    "C:/Users/martt/PycharmProjects/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")

In [ ]:
from rdflib import Graph
import codecs
import os

In [ ]:
input_graph = Graph()
input_graph2 = Graph()
input_graph3 = Graph()
inference_graph = Graph()

input_file = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu.nt")
input_file2 = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")

input_file3 = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0_Publication0.nt")

inference_file = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/jena_inference_with_descriptions/HTTP_www.Department0.University0.edu.nt")

input_graph.parse(data=codecs.open(input_file, encoding="UTF-8").read(), format="nt")
inference_graph.parse(data=codecs.open(inference_file, encoding="UTF-8").read(), format="nt")
input_graph2.parse(data=codecs.open(input_file2, encoding="UTF-8").read(), format="nt")
input_graph3.parse(data=codecs.open(input_file3, encoding="UTF-8").read(), format="nt")
graph = input_graph

In [ ]:
triples_list = sorted(list(input_graph))[0]

In [ ]:
from graph_words_encoder import GraphWordsEncoder
from prepare_data_utils import get_properties_dict, get_classes_dict, get_properties_groups

In [ ]:
import configparser
import json
import logging

In [ ]:
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

config = configparser.ConfigParser()
with open('../config.json', 'r') as f:
    config = json.load(f)

In [ ]:
LUBM_ONTOLOGY_ENDPOINT = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT']
LUBM_ONTOLOGY_REPOSITORY = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_REPOSITORY']
LUBM_ONTOLOGY_ENDPOINT_TYPE = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT_TYPE']
PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['PROPERTIES_QUERY']
QUERY_LIMIT = config['LUBM']['SPARQL_PARAMS']['QUERY_LIMIT']
PROPERTIES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_DICT_FILE']
CLASSES_QUERY = config['LUBM']['SPARQL_PARAMS']['CLASSES_QUERY']
LUBM_ENCODING_DIRECTORY = config['LUBM']['ENCODING_PARAMS']['ENCODING_DIRECTORY']
CLASSES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['CLASSES_DICT_FILE']
PROPERTIES_GROUPS_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_GROUPS_FILE']
GRAPH_WORDS_ENCODER = config['LUBM']['ENCODING_PARAMS']['GRAPH_WORDS_ENCODER']
SUB_PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['SUB_PROPERTIES_QUERY']
INPUT_GRAPHS_FOLDER = config['LUBM']['DATA']['INPUT_GRAPHS_FOLDER']
INFERENCE_GRAPHS_FOLDER = config['LUBM']['DATA']['INFERENCE_GRAPHS_FOLDER']
DATASET_ENCODING_FILE = config['LUBM']['ENCODING_PARAMS']['DATASET_ENCODING_FILE']
INPUT_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INPUT_GRAPH_WORDS_CATALOGUE_FILE']
INFERENCE_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INFERENCE_GRAPH_WORDS_CATALOGUE_FILE']
EMBEDDING_MATRIX_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['EMBEDDING_MATRIX_CATALOGUE_FILE']
MAXIMUM_MATRIX_SIZE = config['LUBM']['ENCODING_PARAMS']['MAXIMUM_MATRIX_SIZE']
HOPE_EMBEDDING_SIZE = config['LUBM']['ENCODING_PARAMS']['HOPE_EMBEDDING_SIZE']

In [ ]:
properties_dict = get_properties_dict(LUBM_ENCODING_DIRECTORY + PROPERTIES_DICT_FILE, PROPERTIES_QUERY,
                                      LUBM_ONTOLOGY_REPOSITORY, LUBM_ONTOLOGY_ENDPOINT,
                                      LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [ ]:
classes_dict = get_classes_dict(LUBM_ENCODING_DIRECTORY + CLASSES_DICT_FILE, CLASSES_QUERY, LUBM_ONTOLOGY_REPOSITORY,
                                LUBM_ONTOLOGY_ENDPOINT,
                                LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [ ]:
properties_group_dict = get_properties_groups(LUBM_ENCODING_DIRECTORY + PROPERTIES_GROUPS_FILE, SUB_PROPERTIES_QUERY,
                                              LUBM_ONTOLOGY_REPOSITORY,
                                              LUBM_ONTOLOGY_ENDPOINT,
                                              LUBM_ONTOLOGY_ENDPOINT_TYPE, properties_dict, QUERY_LIMIT)

In [ ]:
for key in properties_dict.getKeys():
    print(key)

In [ ]:
lubm_graph_words_encoder = GraphWordsEncoder(properties_dict, properties_group_dict, classes_dict)

In [ ]:
input_graph_encoding, resources_dictionary = lubm_graph_words_encoder.encode_graph(input_graph)

In [ ]:
resources_dictionary.keys()

In [ ]:
inference_graph_encoding, resources_dictionary = lubm_graph_words_encoder.encode_graph(inference_graph, inference=True)

In [ ]:
resources_dictionary.keys()

In [ ]:
for key in properties_dict.getKeys():
    print(key)

In [ ]:
for key in properties_group_dict.keys():  #%%
    import pickle
import zlib

import pandas as pd


def deserialize(pz):
    if os.path.isfile(pz):
        compressed = open(pz, 'rb').read()
        pkl = zlib.decompress(compressed)
        tpl = pickle.loads(pkl)
        return tpl
    else:
        return []


lubm_classes_file = "../data/lubm1_intact/encoding/lubm_classes.pz"
lubm_properties_file = "../data/lubm1_intact/encoding/lubm_properties.pz"
lubm_properties_groups_file = "../data/lubm1_intact/encoding/lubm_properties_groups.pz"
tbox_file = "../data/lubm1_intact/encoding/tbox.nt"

lubm_classes = deserialize(lubm_classes_file)
lubm_properties = deserialize(lubm_properties_file)
lubm_properties_groups = deserialize(lubm_properties_groups_file)

data = []

for key, value in lubm_classes.items():
    data.append({"type": "class", "uri": key, "index": value})

for key, value in lubm_properties.items():
    data.append({"type": "property", "uri": key, "index": value})

for key, value in lubm_properties_groups.items():
    data.append({"type": "property_group", "uri": key, "index": value})

df = pd.DataFrame(data)
print(df[df.type == "class"].size,
      df[df.type == "property"].size,
      df[df.type == "property_group"].size)

#lubm_classes
df.shape

# Creating a DF with clases, properties, property_groups
Even though relative - there is no relation between index

In [ ]:
result_df = df.pivot_table(index='index', columns='type', values='uri', aggfunc='first')

In [ ]:
result_df

In [ ]:
df_class = df[df.type == "class"]
df_property = df[df.type == "property"]
df_property_group = df[df.type == "property_group"]

In [ ]:
col_names = ["subject", "predicate", "object"]
#df2 = pd.read_csv(tbox_file, delimiter=' ', names=col_names)
df2 = pd.read_csv(tbox_file, delimiter=' ')
#df2['object'] = df2['object'].apply(lambda x: x.rstrip('.'))
df2 = df2.drop(df2.columns[3], axis=1)
df2.columns = col_names
df2.shape

# I want to see what "property types" are in the #type in df2 for "df1" values marked as "property"

In [ ]:
df_class

In [ ]:
df_property_group

# I will seperate only the subject #end part and compare

In [ ]:
df_property_copy = df_property.copy()

In [ ]:
df_property_copy.uri = df_property_copy.uri.str.extract(r'#(.+)')

In [ ]:
df_property_copy

In [ ]:
df1_property_list = df_property_copy.uri.to_list()

# DF 2 CLASSES

In [ ]:
df2

In [ ]:
df2_types = df2[df2.predicate.str.contains('#type')]

In [ ]:
df2_classes = df2[df2.object.str.contains('#Class')]

In [ ]:
df2_classes

# DF 2 PROPERTIES

In [ ]:
df2_types

# Compare property list from df1 to df2 subject, and get object "type"

In [ ]:
df1_property_list

In [ ]:
types = []
for x in range(len(df1_property_list)):
    types.append(df2_types[df2_types.subject.str.contains(df1_property_list[x])].object.str.extract(r'#(.+)'))

In [ ]:
types_2 = []
for type in types:
    types_2.append(type.to_string().strip())

In [ ]:
types_2

In [ ]:
df2_properties = df2_types.copy()

In [ ]:
df2_properties = df2_properties[df2_properties.object.str.contains("Property")].subject.unique()

In [ ]:
df2_properties

# DF2 SUB PROPERTIES

In [ ]:
df2_sub_properties = df2.copy()

In [ ]:
df2_sub_properties = df2_sub_properties[df2_sub_properties.predicate.str.contains("subPropertyOf")]

In [ ]:
df2_sub_properties

In [ ]:
import re


def get_lubm_graph_type(s):
    s = s.split('/')[-1]
    print(s)
    s = s.split('_')[-1]
    print(s)
    m = re.search("\d", s)
    print(m)
    return s[:m.start()]

In [ ]:
get_lubm_graph_type("HTTP_www.Department0.University0.edu_AssistantProfessor8_Publication1.nt")


In [ ]:
get_lubm_graph_type(
    "C:/Users/martt/PycharmProjects/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")

In [ ]:
from rdflib import Graph
import codecs
import os

In [ ]:
input_graph = Graph()
input_graph2 = Graph()
input_graph3 = Graph()
inference_graph = Graph()

input_file = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu.nt")
input_file2 = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")

input_file3 = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0_Publication0.nt")

inference_file = os.path.abspath(
    "/home/mart/git/NMT4RDFS/data/lubm1_intact/jena_inference_with_descriptions/HTTP_www.Department0.University0.edu.nt")

input_graph.parse(data=codecs.open(input_file, encoding="UTF-8").read(), format="nt")
inference_graph.parse(data=codecs.open(inference_file, encoding="UTF-8").read(), format="nt")
input_graph2.parse(data=codecs.open(input_file2, encoding="UTF-8").read(), format="nt")
input_graph3.parse(data=codecs.open(input_file3, encoding="UTF-8").read(), format="nt")
graph = input_graph

In [ ]:
triples_list = sorted(list(input_graph))[0]

In [ ]:
from graph_words_encoder import GraphWordsEncoder
from prepare_data_utils import get_properties_dict, get_classes_dict, get_properties_groups

In [ ]:
import configparser
import json
import logging

In [ ]:
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

config = configparser.ConfigParser()
with open('../config.json', 'r') as f:
    config = json.load(f)

In [ ]:
LUBM_ONTOLOGY_ENDPOINT = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT']
LUBM_ONTOLOGY_REPOSITORY = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_REPOSITORY']
LUBM_ONTOLOGY_ENDPOINT_TYPE = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT_TYPE']
PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['PROPERTIES_QUERY']
QUERY_LIMIT = config['LUBM']['SPARQL_PARAMS']['QUERY_LIMIT']
PROPERTIES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_DICT_FILE']
CLASSES_QUERY = config['LUBM']['SPARQL_PARAMS']['CLASSES_QUERY']
LUBM_ENCODING_DIRECTORY = config['LUBM']['ENCODING_PARAMS']['ENCODING_DIRECTORY']
CLASSES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['CLASSES_DICT_FILE']
PROPERTIES_GROUPS_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_GROUPS_FILE']
GRAPH_WORDS_ENCODER = config['LUBM']['ENCODING_PARAMS']['GRAPH_WORDS_ENCODER']
SUB_PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['SUB_PROPERTIES_QUERY']
INPUT_GRAPHS_FOLDER = config['LUBM']['DATA']['INPUT_GRAPHS_FOLDER']
INFERENCE_GRAPHS_FOLDER = config['LUBM']['DATA']['INFERENCE_GRAPHS_FOLDER']
DATASET_ENCODING_FILE = config['LUBM']['ENCODING_PARAMS']['DATASET_ENCODING_FILE']
INPUT_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INPUT_GRAPH_WORDS_CATALOGUE_FILE']
INFERENCE_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INFERENCE_GRAPH_WORDS_CATALOGUE_FILE']
EMBEDDING_MATRIX_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['EMBEDDING_MATRIX_CATALOGUE_FILE']
MAXIMUM_MATRIX_SIZE = config['LUBM']['ENCODING_PARAMS']['MAXIMUM_MATRIX_SIZE']
HOPE_EMBEDDING_SIZE = config['LUBM']['ENCODING_PARAMS']['HOPE_EMBEDDING_SIZE']

In [ ]:
properties_dict = get_properties_dict(LUBM_ENCODING_DIRECTORY + PROPERTIES_DICT_FILE, PROPERTIES_QUERY,
                                      LUBM_ONTOLOGY_REPOSITORY, LUBM_ONTOLOGY_ENDPOINT,
                                      LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [ ]:
classes_dict = get_classes_dict(LUBM_ENCODING_DIRECTORY + CLASSES_DICT_FILE, CLASSES_QUERY, LUBM_ONTOLOGY_REPOSITORY,
                                LUBM_ONTOLOGY_ENDPOINT,
                                LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [ ]:
properties_group_dict = get_properties_groups(LUBM_ENCODING_DIRECTORY + PROPERTIES_GROUPS_FILE, SUB_PROPERTIES_QUERY,
                                              LUBM_ONTOLOGY_REPOSITORY,
                                              LUBM_ONTOLOGY_ENDPOINT,
                                              LUBM_ONTOLOGY_ENDPOINT_TYPE, properties_dict, QUERY_LIMIT)

In [ ]:
for key in properties_dict.getKeys():
    print(key)

In [ ]:
lubm_graph_words_encoder = GraphWordsEncoder(properties_dict, properties_group_dict, classes_dict)

In [ ]:
input_graph_encoding, resources_dictionary = lubm_graph_words_encoder.encode_graph(input_graph)

In [ ]:
resources_dictionary.keys()

In [ ]:
inference_graph_encoding, resources_dictionary = lubm_graph_words_encoder.encode_graph(inference_graph, inference=True)

In [ ]:
resources_dictionary.keys()

In [65]:
for key in properties_dict.getKeys():
    print(key, properties_dict[key])

http://www.w3.org/1999/02/22-rdf-syntax-ns#type 1
http://swat.cse.lehigh.edu/onto/univ-bench.owl#advisor 2
http://swat.cse.lehigh.edu/onto/univ-bench.owl#affiliateOf 3
http://swat.cse.lehigh.edu/onto/univ-bench.owl#affiliatedOrganizationOf 4
http://swat.cse.lehigh.edu/onto/univ-bench.owl#age 5
http://swat.cse.lehigh.edu/onto/univ-bench.owl#degreeFrom 6
http://swat.cse.lehigh.edu/onto/univ-bench.owl#doctoralDegreeFrom 7
http://swat.cse.lehigh.edu/onto/univ-bench.owl#emailAddress 8
http://swat.cse.lehigh.edu/onto/univ-bench.owl#hasAlumnus 9
http://swat.cse.lehigh.edu/onto/univ-bench.owl#headOf 10
http://swat.cse.lehigh.edu/onto/univ-bench.owl#listedCourse 11
http://swat.cse.lehigh.edu/onto/univ-bench.owl#mastersDegreeFrom 12
http://swat.cse.lehigh.edu/onto/univ-bench.owl#member 13
http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf 14
http://swat.cse.lehigh.edu/onto/univ-bench.owl#name 15
http://swat.cse.lehigh.edu/onto/univ-bench.owl#orgPublication 16
http://swat.cse.lehigh.edu/onto

In [66]:
for key in classes_dict.getKeys():
    print(key, classes_dict[key])

http://swat.cse.lehigh.edu/onto/univ-bench.owl#AdministrativeStaff 1
http://swat.cse.lehigh.edu/onto/univ-bench.owl#Article 2
http://swat.cse.lehigh.edu/onto/univ-bench.owl#AssistantProfessor 3
http://swat.cse.lehigh.edu/onto/univ-bench.owl#AssociateProfessor 4
http://swat.cse.lehigh.edu/onto/univ-bench.owl#Book 5
http://swat.cse.lehigh.edu/onto/univ-bench.owl#Chair 6
http://swat.cse.lehigh.edu/onto/univ-bench.owl#ClericalStaff 7
http://swat.cse.lehigh.edu/onto/univ-bench.owl#College 8
http://swat.cse.lehigh.edu/onto/univ-bench.owl#ConferencePaper 9
http://swat.cse.lehigh.edu/onto/univ-bench.owl#Course 10
http://swat.cse.lehigh.edu/onto/univ-bench.owl#Dean 11
http://swat.cse.lehigh.edu/onto/univ-bench.owl#Department 12
http://swat.cse.lehigh.edu/onto/univ-bench.owl#Director 13
http://swat.cse.lehigh.edu/onto/univ-bench.owl#Employee 14
http://swat.cse.lehigh.edu/onto/univ-bench.owl#Faculty 15
http://swat.cse.lehigh.edu/onto/univ-bench.owl#FullProfessor 16
http://swat.cse.lehigh.edu/onto

In [67]:
for key in properties_group_dict.keys():
    print(key, properties_group_dict[key])


http://www.w3.org/2000/01/rdf-schema#seeAlso 0
http://www.w3.org/2000/01/rdf-schema#isDefinedBy 0
http://www.w3.org/1999/02/22-rdf-syntax-ns#_1 1
http://www.w3.org/2000/01/rdf-schema#member 1
http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf 2
http://swat.cse.lehigh.edu/onto/univ-bench.owl#headOf 2
http://swat.cse.lehigh.edu/onto/univ-bench.owl#worksFor 2
http://swat.cse.lehigh.edu/onto/univ-bench.owl#doctoralDegreeFrom 3
http://swat.cse.lehigh.edu/onto/univ-bench.owl#undergraduateDegreeFrom 3
http://swat.cse.lehigh.edu/onto/univ-bench.owl#degreeFrom 3
http://swat.cse.lehigh.edu/onto/univ-bench.owl#mastersDegreeFrom 3
http://www.w3.org/1999/02/22-rdf-syntax-ns#type 4
http://swat.cse.lehigh.edu/onto/univ-bench.owl#advisor 5
http://swat.cse.lehigh.edu/onto/univ-bench.owl#affiliateOf 6
http://swat.cse.lehigh.edu/onto/univ-bench.owl#affiliatedOrganizationOf 7
http://swat.cse.lehigh.edu/onto/univ-bench.owl#age 8
http://swat.cse.lehigh.edu/onto/univ-bench.owl#emailAddress 9
http://swat